# Bases de datos en LangChain.

En este apartado vamos a ver los siguientes aspectos:

* ¿Cómo cargar fuentes de datos de todo tipo con Langchain

* ¿Cómo transformar documentos y fragmentarlos?

* ¿Cómo convertir documentos en vectores a partir de incrustaciones embeddings

* ¿Cómo almacenar los datos (internos y externos) en una base de datos vectorizada?

* ¿Cómo realizar consultas a la base de datos vectorizada y mejorar los resultados con LLMs

Comenzamos con el primer apartado, es decír, cómo poder cargar datos de múltiples fuentes.

## Cargadores de documentos.

Langchain viene con herramientas de carga integradas para cargar rápidamente archivos en su propio objeto Documento.

Muchos de estos cargadores requieren otras bibliotecas, por ejemplo, la carga de PDF requiere la biblioteca pypdf y la carga de HTML requiere la biblioteca Beautiful Soup . Asegurar de instalar las bibliotecas requeridas antes de usar el cargador (los cargadores informarán si no pueden encontrar las bibliotecas instaladas).

Entre otras librerias es muy conveniente tener instalada la  librería *Langchain community* para loaders en Python .

Para ver la documentación sobre los loaders de LangChain, se puede visitar el siguiente enlace: 

https://python.langchain.com/v0.2/docs/integrations/document_loaders/

Procedemos a cargar las librerías e instanciar el modelo de tipo de chat

In [1]:
import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate

chat = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

## Cargar documentos de tipo CSV

In [2]:
from langchain.document_loaders import CSVLoader #pip install langchain-community en una terminal

In [4]:
#Cargamos el fichero CSV
loader = CSVLoader('Fuentes datos/datos_ventas_small.csv',csv_args={'delimiter': ';'})
#Creamos el objeto "data" con los datos desde el cargador "loader"
data = loader.load()
#print(data) #Vemos que se ha creado un documento por cada fila donde el campo page_content contiene los datos
data[0]

Document(metadata={'source': 'Fuentes datos/datos_ventas_small.csv', 'row': 0}, page_content='ï»¿ID: 10145\nCantidad: 45\nPrecio unitario: 83,26\nVenta total: 3746,7\nFecha compra: 25/08/2023\nEstado: Shipped\nLÃ\xadnea Producto: Motorcycles\nCÃ³digo Producto: S10_1678\nNombre cliente: Toys4GrownUps,com\nCiudad: Pasadena\nPaÃ\xads: USA\nTerritorio: NA\nTamaÃ±o pedido: Medium')

In [5]:
print(data[1].page_content)

ï»¿ID: 10159
Cantidad: 0
Precio unitario: 100
Venta total: 0
Fecha compra: 10/10/2023
Estado: Shipped
LÃ­nea Producto: Motorcycles
CÃ³digo Producto: S10_1678
Nombre cliente: Corporate Gift Ideas Co,
Ciudad: San Francisco
PaÃ­s: USA
Territorio: NA
TamaÃ±o pedido: Medium


## Cargar datos HTML

In [6]:
from langchain.document_loaders import BSHTMLLoader #pip install beautifulsoup4 en una terminal

In [7]:
loader = BSHTMLLoader('Fuentes datos/ejemplo_web.html')
data = loader.load()
data

[Document(metadata={'source': 'Fuentes datos/ejemplo_web.html', 'title': ''}, page_content='\n\n\n\n\nSQL, Structure Query Language (Lenguaje de Consulta Estructurado) es un lenguaje de\nprogramacion para trabajar con base de datos relacionales como MySQL, Oracle, etc.\nMySQL es un interpretador de SQL, es un servidor de base de datos.\nMySQL permite crear base de datos y tablas, insertar datos, modificarlos, eliminarlos,\nordenarlos, hacer consultas y realizar muchas operaciones, etc., resumiendo: administrar bases\nde datos.\n\n\nEste tutorial tiene por objetivo acercar los conceptos iniciales para introducirse en el mundo de\nlas bases de datos.\n\n\n\n')]

In [8]:
print(data[0].page_content)






SQL, Structure Query Language (Lenguaje de Consulta Estructurado) es un lenguaje de
programacion para trabajar con base de datos relacionales como MySQL, Oracle, etc.
MySQL es un interpretador de SQL, es un servidor de base de datos.
MySQL permite crear base de datos y tablas, insertar datos, modificarlos, eliminarlos,
ordenarlos, hacer consultas y realizar muchas operaciones, etc., resumiendo: administrar bases
de datos.


Este tutorial tiene por objetivo acercar los conceptos iniciales para introducirse en el mundo de
las bases de datos.






## Cargar datos PDF

In [9]:
from langchain.document_loaders import PyPDFLoader #pip install pypdf en una terminal

In [10]:
loader = PyPDFLoader('Fuentes datos/Documento tecnologías emergentes.pdf')
pages = loader.load_and_split()
type(pages)

list

In [11]:
pages[0]

Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2021-01-16T17:12:17+01:00', 'author': 'Ivan Pinar', 'moddate': '2021-01-16T17:12:17+01:00', 'source': 'Fuentes datos/Documento tecnologías emergentes.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Estas son las 9 tecnologías \nemergentes para el próximo \n2025  \n  \n“Que la tecnología ha cambiado nuestra manera de vivir e interactuar \nes un hecho. Sin embargo, aún no somos conscientes de las \npotencialidades de usos de las tecnologías.Por  ejemplo, para el año \n2025 se espera una verdadera revolución tecnológica, sobre todo \nenfocado en el  sector bio -médico  pero también en las relaciones \nhumanas entre individuos a distancia, en la protección del medio \nambiente o en la protección de nuestros d atos personales ”, afirma \nJuan Quintanilla, director general de Syntonize.  \n9 Tecnologías emergentes según  Syntonize  \nLa aplicación de nuevas tecnolog

In [12]:
print(pages[0].page_content)

Estas son las 9 tecnologías 
emergentes para el próximo 
2025  
  
“Que la tecnología ha cambiado nuestra manera de vivir e interactuar 
es un hecho. Sin embargo, aún no somos conscientes de las 
potencialidades de usos de las tecnologías.Por  ejemplo, para el año 
2025 se espera una verdadera revolución tecnológica, sobre todo 
enfocado en el  sector bio -médico  pero también en las relaciones 
humanas entre individuos a distancia, en la protección del medio 
ambiente o en la protección de nuestros d atos personales ”, afirma 
Juan Quintanilla, director general de Syntonize.  
9 Tecnologías emergentes según  Syntonize  
La aplicación de nuevas tecnologías que hagan más fácil la vida a 
profesionales, estudiantes, mayores, empresas o instituciones 
públicas se e spera que aumente en los próximos años. Entre ellas se 
encuentran;  
 Producción optimizada por la Inteligencia Artificial:  las 
empresas están adoptando rápidamente tecnologías basadas 
en la nube. Gracias a ello, podrán ag

## Caso de uso resumir un documento

En este apartado vamos a ver un ejemplo concreto sobre como poder utilizar el poder la IA para hacer un resumen de un texto

In [14]:
contenido_pdf=pages[0].page_content
contenido_pdf

'Estas son las 9 tecnologías \nemergentes para el próximo \n2025  \n  \n“Que la tecnología ha cambiado nuestra manera de vivir e interactuar \nes un hecho. Sin embargo, aún no somos conscientes de las \npotencialidades de usos de las tecnologías.Por  ejemplo, para el año \n2025 se espera una verdadera revolución tecnológica, sobre todo \nenfocado en el  sector bio -médico  pero también en las relaciones \nhumanas entre individuos a distancia, en la protección del medio \nambiente o en la protección de nuestros d atos personales ”, afirma \nJuan Quintanilla, director general de Syntonize.  \n9 Tecnologías emergentes según  Syntonize  \nLa aplicación de nuevas tecnologías que hagan más fácil la vida a \nprofesionales, estudiantes, mayores, empresas o instituciones \npúblicas se e spera que aumente en los próximos años. Entre ellas se \nencuentran;  \n\uf0b7 Producción optimizada por la Inteligencia Artificial:  las \nempresas están adoptando rápidamente tecnologías basadas \nen la nube. 

In [15]:
human_template = '"Necesito que hagas un resumen del siguiente texto: \n{contenido}"'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [16]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

chat_prompt.format_prompt(contenido=contenido_pdf)

ChatPromptValue(messages=[HumanMessage(content='"Necesito que hagas un resumen del siguiente texto: \nEstas son las 9 tecnologías \nemergentes para el próximo \n2025  \n  \n“Que la tecnología ha cambiado nuestra manera de vivir e interactuar \nes un hecho. Sin embargo, aún no somos conscientes de las \npotencialidades de usos de las tecnologías.Por  ejemplo, para el año \n2025 se espera una verdadera revolución tecnológica, sobre todo \nenfocado en el  sector bio -médico  pero también en las relaciones \nhumanas entre individuos a distancia, en la protección del medio \nambiente o en la protección de nuestros d atos personales ”, afirma \nJuan Quintanilla, director general de Syntonize.  \n9 Tecnologías emergentes según  Syntonize  \nLa aplicación de nuevas tecnologías que hagan más fácil la vida a \nprofesionales, estudiantes, mayores, empresas o instituciones \npúblicas se e spera que aumente en los próximos años. Entre ellas se \nencuentran;  \n\uf0b7 Producción optimizada por la In

In [17]:
solicitud_completa = chat_prompt.format_prompt(contenido=contenido_pdf).to_messages()
result = chat.invoke(solicitud_completa)
result.content

'Estás leyendo un artículo sobre las 9 tecnologías emergentes esperadas para el año 2025 según Juan Quintanilla, director general de Syntonize. A continuación, te presento un resumen del texto:\n\nSegún Juan Quintanilla, la tecnología está cambiando nuestra forma de vida y interactuar con otros, pero ainda no estamos conscientes de las posibles aplicaciones y usos de estas tecnologías.\n\nEn cuanto a las 9 tecnologías emergentes esperadas para 2025, se incluyen:\n\n1. Producción optimizada por la Inteligencia Artificial\n2. Tecnologías basadas en la nube (donde se agrega valor al flujo de datos y los algoritmos inteligentes)\n3. Otro punto no se menciona en esta parte del texto.\n\nNo se proporciona una lista completa de las 9 tecnologías, ya que solo se mencionan dos en el texto fornecido: la producción optimizada por la Inteligencia Artificial y la aplicación de la nube.'

In [18]:
#Resumir el documento completo
#Creamos una string concatenando el contenido de todas las páginas
documento_completo = ""
for page in pages:
    documento_completo += page.page_content  # Supongamos que cada página tiene un atributo 'text'

print(documento_completo)

Estas son las 9 tecnologías 
emergentes para el próximo 
2025  
  
“Que la tecnología ha cambiado nuestra manera de vivir e interactuar 
es un hecho. Sin embargo, aún no somos conscientes de las 
potencialidades de usos de las tecnologías.Por  ejemplo, para el año 
2025 se espera una verdadera revolución tecnológica, sobre todo 
enfocado en el  sector bio -médico  pero también en las relaciones 
humanas entre individuos a distancia, en la protección del medio 
ambiente o en la protección de nuestros d atos personales ”, afirma 
Juan Quintanilla, director general de Syntonize.  
9 Tecnologías emergentes según  Syntonize  
La aplicación de nuevas tecnologías que hagan más fácil la vida a 
profesionales, estudiantes, mayores, empresas o instituciones 
públicas se e spera que aumente en los próximos años. Entre ellas se 
encuentran;  
 Producción optimizada por la Inteligencia Artificial:  las 
empresas están adoptando rápidamente tecnologías basadas 
en la nube. Gracias a ello, podrán ag

In [19]:
solicitud_completa = chat_prompt.format_prompt(contenido=documento_completo).to_messages()
result = chat.invoke(solicitud_completa)

In [20]:
result.content

'A continuación, te presento un resumen del texto sobre 9 tecnologías emergentes para el año 2025:\n\n1. **Producción optimizada por la Inteligencia Artificial**: la implementación de tecnologías basadas en IA permitirá agregar valor a datos de productos y procesos de las líneas de fabricación, reduciendo el desperdicio total en la fabricación hasta un 50%.\n\n2. **Energías renovables de largo alcance**: se espera que la huella de carbono sea socialmente inaceptable en 2025, lo que impulsará una industria masiva para gestionar el carbono y capturar, utilizar y eliminar el dióxido de carbono.\n\n3. **Ordenadores cuánticos**: estos ordenadores podrán abordar problemas complejos como reacciones químicas y facilitar la investigación y la aplicación médica.\n\n4. **Prevención sanitaria a través de la comida**: se espera que los sistemas de atención médica adopten enfoques de salud más preventivos basados en la ciencia detrás de los beneficios para la salud de las dietas ricas en nutrientes 

## Integración con otras plataformas.

Existen otros cargadores de documentos que son denominados "integraciones" y pueden ser considerados esencialmente lo mismo que los cargadores normales vistos en la sección anterior, pero con la salvedad y la ventaja de que están integrados con otras plataformas como por ejemplo:

* Plataforma de terceros (como Google Cloud, AWS, Google Drive, Dropbox,…)

* Base de datos (como MongoDB)

* Sitio web específico, como Wikipedia

* Permiten cargar vídeos de Youtube (por ejemplo, crear una aplicación de preguntas y respuestas en base a vídeos de Youtube ), conversaciones de WhatsApp y un sinfín de posibilidades.

Con todas estas integraciones, vamos a tener la ventaja de cargar esta información en una base de datos vectorial y después consultar esa información con todas las ventajas que esta información nos puede proporcionar.


La documentación sobre este tipo de cargadores (document loaders - integraciones), se tiene en este enlace: 

https://python.langchain.com/v0.2/docs/integrations/document_loaders/

### Cargar informaciones de wikipedia.

A continuación vamos a mostrar un caso de usos que consiste en cargar información de la wikipedia. Como siempre cargamos los paquetes correspondientes y cargamos el chat.

In [1]:
import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate


chat = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

In [6]:
#!pip install wikipedia

In [2]:
from langchain.document_loaders import WikipediaLoader # pip install wikipedia en una terminal

Definimos la siguiente función que es la que nos va a servir para obtener y ejecutar lo que necesitamos para hacer consultas apoyadas en la información que figura en la wikipedia.

In [3]:
def responder_wikipedia(persona,pregunta_arg):
    # Obtener artículo de wikipedia
    docs = WikipediaLoader(query=persona,lang="es",load_max_docs=10) #parámetros posibles en: https://python.langchain.com/v0.2/docs/integrations/document_loaders/wikipedia/
    # Observar que el valor de "persona" lo pasamos como parámetro a la función
    contexto_extra = docs.load()[0].page_content #para que sea más rápido solo pásamos el primer documento [0] como contexto extra
    
    # Pregunta de usuario, que se la pasamos como parámetro de la función
    human_prompt = HumanMessagePromptTemplate.from_template('Responde a esta pregunta\n{pregunta}, aquí tienes contenido extra:\n{contenido}')
    
    # Construir prompt
    chat_prompt = ChatPromptTemplate.from_messages([human_prompt])
    
    # Resultado
    result = chat.invoke(chat_prompt.format_prompt(pregunta=pregunta_arg,contenido=contexto_extra).to_messages())
    
    print(result.content)

In [9]:
responder_wikipedia("José María Aznar","¿En qué localidad nació?")

Según la información proporcionada, José María Aznar López nació en Madrid, España.


## Transformación de documentos.
```{index} transformadores de documentos, chunks
```

Hay que tener en cuenta que después de cargar un objeto Documento desde una fuente, terminará con cadenas de texto desde el campo page_content. Entonces puede haber situaciones en las que la longitud de las cadenas asó obtenidas pueden ser muy grandes  para alimentar un modelo (por ejemplo, límite de 8k tokens ~6k palabras). Para resolver este problema, Langchain proporciona **transformadores de documentos** que permiten dividir fácilmente cadenas del page_content en fragmentos (que se conocen como chunks).

Estos fragmentos servirán más adelante además como componentes útiles en forma de vectores a partir de una incrustación (embeddings ), que luego podremos buscar utilizando una similitud de distancia más adelante. Por ejemplo, si queremos alimentar un LLM con contexto adicional para que sirva como chatbot de preguntas y respuestas, si tenemos varios vectores guardados cada uno con una información diferente, la búsqueda será más rápida puesto que se hará un cálculo del vector guardado que tiene mayor similaridad en lugar de buscar en todos los datos globales.

Veamos ahora un ejemplo ilustrativo de cómo poder hacer todo esto. Para hacer esto vamos a cargar un documento bastante extenso y con mucha información.

In [11]:
with open('Fuentes datos/Historia España.txt', encoding="utf8") as file:
    texto_completo = file.read()

# Números de caracteres
len(texto_completo)

85369

Como podemos ver es un documento bastante extenso y lo que vamos a hacer es dividirlo en trozos más pequeños, los cuales tienen la denominación de chunks.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [13]:
text_splitter = CharacterTextSplitter(separator="\n",chunk_size=1000) #Indicamos que divida cuando se encuentra 1 salto de línea y trate de hacer fragmentos de 1000 caracteres
# Intenta hacer los chunks más o menos del tamaño que se le da, en este caso de 1000

Existen muchas más posibilidades para hacer esto, las cuales se pueden ver en este enlace: 
https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.CharacterTextSplitter.html

Entre estas posibilidaddes está una muy utilizada y es que los cuhunks puedan tener cierto solpamientos, es decir que las últimas palabras del chunk anterior, sean también las palabras del chunk siguiente. Este efecto lo conseguimos con la opción *chunk_overlap*.

In [14]:
texts = text_splitter.create_documents([texto_completo]) #Creamos documentos gracias al transformador

Created a chunk of size 1424, which is longer than the specified 1000
Created a chunk of size 1290, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1232, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000
Created a chunk of size 1053, which is longer than the specified 1000
Created a chunk of size 1248, which is longer than the specified 1000


In [15]:
print(type(texts)) #Verificamos el tipo del objeto obtenido
print('\n')
print(type(texts[0])) #Verificamos el tipo de cada elemento
print('\n')
print(texts[0])

<class 'list'>


<class 'langchain_core.documents.base.Document'>


page_content='Los primeros homínidos llegaron al territorio de la actual España hace 1,2 millones de años aproximadamente. Se sucedieron varias especies, como Homo antecessor, los preneandertales de la Sima de los Huesos (identificados en un principio como Homo heidelbergensis) y los neandertales (Homo neanderthalensis), hasta que hace unos 35 000 años los humanos modernos (Homo sapiens) entraron en la península ibérica y fueron desplazando a estos últimos, con los que aún coexistirían durante cerca de 10 000 años. Hace unos 27 000 años se extinguieron las últimas poblaciones neandertales en el sur. Durante los milenios siguientes el territorio fue lugar del asentamiento de pueblos íberos, celtas, fenicios, cartagineses, tartessos y griegos y hacia el 200 a. C. la península comenzó a formar parte de la República romana, constituyendo la Hispania romana. Tras la caída de Roma, se estableció el reino visigodo. Dicha mona

In [16]:
len(texts[0].page_content)

1424

In [17]:
texts[1]

Document(metadata={}, page_content='De modo paulatino, se produjo la Reconquista, y los reinos cristianos arrebataron progresivamente el territorio a los musulmanes. Comenzada aproximadamente en 722 con la rebelión de Don Pelayo y partiendo desde el norte, avanzó durante los siglos viii a xv culminando con la conquista de Granada en 1492. Durante este periodo los reinos cristianos se desarrollaron notablemente; la unión de los dos más importantes, Castilla y Aragón, por el matrimonio en 1469 de los Reyes Católicos, Isabel I de Castilla y Fernando II de Aragón, posibilitaría la unificación de España y el fin de la Reconquista.2\u200b3\u200b4\u200b5\u200b')

In [21]:
# Veamos la longitud de cada uno de los chunks que se han obtenido
for h in texts:
    print(len(h.page_content))

1424
605
581
589
688
696
978
832
836
889
914
687
647
863
811
783
951
957
355
978
687
809
380
965
977
670
692
734
620
878
1290
653
976
910
919
577
758
588
473
940
850
857
272
1191
758
1232
985
949
1193
915
453
903
1000
822
664
984
471
562
801
697
572
824
985
933
558
977
927
993
864
811
949
768
486
995
1053
812
972
943
983
969
975
955
386
1248
658
924
875
951
947
979
994
847
893
661
978
959
845
823
991
811
995
918
890
865
759
878
990
927
728


## Incrustación de texto y creación de vectores (embeging)
```{index} Embeding, OllamaEmbeddings
```

**NOTA**: [En este otro apartado](embeding), también se puede ver desde diferentes puntos de vista cómo poder trabajar con este tipo embeding.

De cara a trabajar con textos en IA, lo que se suele hacer es transformar esos textos en una representación de los mismos mediante una serie de vectores que contienen información semántica de esos textos. Langchain admite muchas incrustaciones de texto, que pueden convertir directamente texto en una representación vectorizada incrustada.

En resumen, los modelos incrustados crean una representación vectorial de un fragmento de texto . Puedes pensar en un vector como una matriz de números que captura el significado semántico del texto. Al representar el texto de esta manera, puede realizar operaciones matemáticas que le permiten hacer cosas como buscar otras partes del texto que tengan un significado más similar.

![](fig/embeding.PNG)

Estos modelos de embeding que utiliza LangChain, se puede ver su explicación en el siguiente enlace:

https://python.langchain.com/v0.2/docs/concepts/#embedding-models

**NOTA** : Los diferentes modelos de incrustación puede que no interactúen entre sí, lo que significa que necesitaría volver a incrustar un conjunto completo de documentos si cambiara de modelo de incrustación en el futuro. En este se indicará cómo utilizar OpenAI, pues es uno de los métodos más utilizados, pero como se intenta hacer una explicación de estos métodos desde un punto de vista didáctico, sin incurrir en costes, se utilizará ollama para hacer cuestiones prácticas sobre estos métodos.

Se aconseja al lector mirar estos enlaces:

* <a href="https://python.langchain.com/docs/integrations/text_embedding/ollama/" target="_blank">Presentación de OllamaEmbeddings </a>

* <a href="https://python.langchain.com/api_reference/ollama/embeddings/langchain_ollama.embeddings.OllamaEmbeddings.html" target="_blank">Api de OllamaEmbeddings </a>

A continuación se muestra un caso práctico sobre cómo utilizar todos estos procesos.

In [1]:
import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate

chat = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

Si quisiéramos hacer esto desde OpenAi, el código a utilizar sería el siguiente: (Se ha dejado comentado el código)

**NOTA**: Para ver cómo empezar con OpenAi, se recomienda ver [este apartado](pago) 

In [2]:
#from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(openai_api_key=api_key)
# texto = "Esto es un texto enviado a OpenAI para ser incrustado en un vector n-dimensional"
#embedded_text = embeddings.embed_query(texto)

Sin embargo y con el fin de evitar costes, vamos a ver cómo haríamos estos embeding, utilizando ollama desde LangChain.

In [3]:
from langchain_ollama import OllamaEmbeddings

In [5]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

texto = "Este es el texto que vamos a vectorizar utilizando para ello llama que sale gratuito"
embedded_text = embeddings.embed_query(texto)

In [6]:
type(embedded_text)

list

In [8]:
embedded_text[:10]

[-0.02119353,
 -0.013681516,
 0.032055188,
 -0.005358407,
 -0.010106426,
 -0.0073281615,
 0.010382513,
 -0.015157732,
 0.009666262,
 -0.0072076097]

In [11]:
# Este mismo ejercicio pero de forma asíncrona
embedded_text = await embeddings.aembed_query(texto)

Si quisiéramos hacer esto con varios textos deberíamos utilizar una expresión similar a la siguiente:

```
input_texts = ["Document 1...", "Document 2..."]
vectors = embed.embed_documents(input_texts)
print(len(vectors))
# The first 3 coordinates for the first vector
print(vectors[0][:3])
```

### Incrustación de documentos.

A continuación se muestra un ejemplo, para ver cómo podemos hacer embedings de documentos, que es la situación real con la que nos encontraremos al trabajar con IA.

Lo primero que hacemos es cargar un documento de tipo CSV


In [12]:
from langchain.document_loaders import CSVLoader

In [13]:
loader = CSVLoader('Fuentes datos/datos_ventas_small.csv',csv_args={'delimiter': ';'})
data = loader.load()
type(data)

list

In [14]:
type(data[0])

langchain_core.documents.base.Document

In [15]:
#No podemos incrustar el objeto "data" puesto que es una lista de documentos, lo que espera es una string
# Ejecutar el siguiente comando nos daría un error
#embedded_docs = embeddings.embed_documents(data)

In [16]:
#Creamos una comprensión de listas concatenando el campo "page_content" de todos los documentos existentes en la lista "data"
[elemento.page_content for elemento in data]

['ï»¿ID: 10145\nCantidad: 45\nPrecio unitario: 83,26\nVenta total: 3746,7\nFecha compra: 25/08/2023\nEstado: Shipped\nLÃ\xadnea Producto: Motorcycles\nCÃ³digo Producto: S10_1678\nNombre cliente: Toys4GrownUps,com\nCiudad: Pasadena\nPaÃ\xads: USA\nTerritorio: NA\nTamaÃ±o pedido: Medium',
 'ï»¿ID: 10159\nCantidad: 0\nPrecio unitario: 100\nVenta total: 0\nFecha compra: 10/10/2023\nEstado: Shipped\nLÃ\xadnea Producto: Motorcycles\nCÃ³digo Producto: S10_1678\nNombre cliente: Corporate Gift Ideas Co,\nCiudad: San Francisco\nPaÃ\xads: USA\nTerritorio: NA\nTamaÃ±o pedido: Medium',
 'ï»¿ID: 10168\nCantidad: 36\nPrecio unitario: 96,66\nVenta total: 3479,76\nFecha compra: 28/10/2023\nEstado: Shipped\nLÃ\xadnea Producto: Motorcycles\nCÃ³digo Producto: S10_1678\nNombre cliente: Technics Stores Inc,\nCiudad: Burlingame\nPaÃ\xads: USA\nTerritorio: NA\nTamaÃ±o pedido: Medium',
 'ï»¿ID: 10180\nCantidad: 29\nPrecio unitario: 86,13\nVenta total: 2497,.77\nFecha compra: 11/11/2023\nEstado: Shipped\nLÃ\xad

In [17]:
embedded_docs = embeddings.embed_documents([elemento.page_content for elemento in data])

In [18]:
#Verificamos cuántos vectores a creado (1 por cada registro del fichero CSV con datos)
len(embedded_docs)

22

In [20]:
#Vemos un ejemplo del vector creado para el primer registro
embedded_docs[1][:10]

[0.006179472,
 0.017413776,
 0.021690493,
 -0.0056598824,
 0.022888422,
 -0.025513476,
 -0.002373873,
 -0.027805334,
 -0.0032582898,
 -0.0102156745]

(almacenamiento)=
## Almacenamiento de vectores en BD.
```{index} BD vectoriales
```

Hasta ahora hemos creado incrustaciones ( embeddings ) en memoria RAM como una lista de Python. Estos embedings en el momento en que nos salgamos de la aplicación se pierden, entonces ¿cómo podemos asegurarnos de que estas incorporaciones persistan en alguna solución de almacenamiento más permanente?

Para conseguir que esta información quede almacenada para futuras consultas, utilizamos un almacén de vectores, también conocido como base de datos de vectores , sus aspectos claves:

* Puede almacenar grandes vectores de N dimensiones.

* Puede indexar directamente un vector incrustado y asociarlo a su documento string

* Se puede "consultar", lo que permite una búsqueda de similitud de coseno entre un nuevo vector que no está en la base de datos y los vectores almacenados.
  
* Puede agregar, actualizar o eliminar fácilmente nuevos vectores.

* Al igual que con los LLM y los modelos de chat, Langchain ofrece muchas opciones diferentes para almacenes de vectores.

* Usaremos una base de datos de vectores open source SKLearn , pero gracias a Langchain , la sintaxis es estándar para el resto de BD.

Para hacer este tipo de persistencia LangChain nos ofrece una amplia variedad de Bases de datos, las cuales las podemos consultar utilizando el siguiente link:

https://python.langchain.com/v0.2/docs/integrations/vectorstores/

La metodología que se emplea para este tipo de persistencia de la información, de forma esquemática se puede ver en la siguiente ilustración:

![](fig/BD.PNG)

Como ya hemos hecho en casos anteriores, y con la finalidad de mostrar como actuar cuando se quiere hacer este tipo de cosas en IA, a continuación se pasa a ilustrar todo esto con algún ejemplo totalmente práctico.

In [21]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

Cargamos el documento y lo dividimos

In [22]:
# Cargar el documento
loader = TextLoader('Fuentes datos/Historia España.txt', encoding="utf8")
documents = loader.load()

# Dividir en chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500) #Otro método de split basándose en tokens
docs = text_splitter.split_documents(documents)

Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 1009, which is longer than the specified 500
Created a chunk of size 2228, which is longer than the specified 500


Procedemos a la creación de embedings

In [24]:
funcion_embedding = OllamaEmbeddings(
    model="llama3.2",
)

Para el almacenamiento, utilizamos *SKLearn Vector Store*

In [25]:
from langchain_community.vectorstores import SKLearnVectorStore #pip install scikit-learn / pip install pandas pyarrow

In [28]:
#!pip install pandas pyarrow

In [29]:
persist_path="./BD/ejemplosk_embedding_db"  #ruta donde se guardará la BBDD vectorizada

#Creamos la BBDD de vectores a partir de los documentos y la función embeddings
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=funcion_embedding,
    persist_path=persist_path,
    serializer="parquet", #el serializador o formato de la BD lo definimos como parquet
)

In [30]:
# Fuerza a guardar los nuevos embeddings en el disco
vector_store.persist()

Una vez ejecutado el anterior código, y apodemos ver en nuestro disco duro la base de datos creada en la carpeta BD y con denominación ejemplosk_embedding_db.

### Búsqueda en la Base de Datos.

Una vez creada la base de datos podremos ya hacer consultas de similitud de cadenas, para que nos encuentre en la BD los párrafos más similares al litereal que le pasamos. Además nos devuelve párrafos ordenados de mayor a menor similitud.

In [31]:
#Creamos un nuevo documento que será nuestra "consulta" para buscar el de mayor similitud en nuestra Base de Datos de Vectores y devolverlo
consulta = "dame información de la Primera Guerra Mundial"
docs = vector_store.similarity_search(consulta)
print(docs[0].page_content)

Dictablanda del general Berenguer (1930-1931)
Artículo principal: Dictablanda de Dámaso Berenguer
Tras la crisis económica de 1927 acentuada en 1929, la violenta represión de obreros e intelectuales y la falta de sintonía entre la burguesía y la dictadura, la monarquía, cómplice, será el objeto en cuestión a partir de la unión de toda la oposición en agosto de 1930 en el llamado Pacto de San Sebastián. Los gobiernos de Dámaso Berenguer, denominado la «dictablanda», y de Juan Bautista Aznar-Cabañas, no harán otra cosa que alargar la decadencia. Tras las elecciones municipales de 1931, el 14 de abril se proclama la Segunda República, dando así fin a la restauración borbónica en España.

Segunda República española (1931-1936)
Artículo principal: Segunda República española


El resultado que obtenemos no es lo que realmente estamos buscando, pero hay que tener en cuenta que estamos trabajando en modo local y con resursos muy limitados debido a los escasos recursos que los ordenadores personales tienen para este tipo de trabajos de IA. Muy posiblemente si esto lo hacemos utilizando el api-key de OpenAi el resultado hubiera sido más acertado y además más rápido

## Recuprar datos de una BD.

Una vez creada la base de datos, y como ya los datos se han persistido y están almacenados en la base de datos, podemos recuperar en cualquier momento la información de esa base de datos y hacer consultas sobre la misma. A continuación se muestra cómo poder hacer esto.

In [32]:
vector_store_connection = SKLearnVectorStore(
    embedding=funcion_embedding, persist_path=persist_path, serializer="parquet"
)
print("Una instancia de la BBDD de vectores se ha cargado desde ", persist_path)

Una instancia de la BBDD de vectores se ha cargado desde  ./BD/ejemplosk_embedding_db


In [33]:
vector_store_connection

In [34]:
nueva_consulta = "¿Qué paso en el siglo de Oro?"
docs = vector_store_connection.similarity_search(nueva_consulta)
print(docs[0].page_content)

Guerra hispano-estadounidense
Artículo principal: Guerra hispano-estadounidense
España pierde Cuba, Filipinas y Puerto Rico: Cuba se rebeló contra España en el comienzo la Guerra de los Diez Años en 1868, dando como resultado la abolición de la esclavitud en las colonias españolas en el Nuevo Mundo. Los intereses estadounidenses en la isla, junto con la preocupación por el pueblo de Cuba, empeoraron las relaciones entre los dos países. La explosión del USS Maine lanzó la guerra de Cuba en 1898, en el que España sufrió un descalabro. Cuba obtuvo su independencia y España perdió sus últimas colonias del Nuevo Mundo: Puerto Rico, junto con Guam y las Filipinas fueron cedidas a los Estados Unidos por 20 millones de dólares. En 1899, España vendió su participación restante de las islas del Pacífico, las islas Marianas del Norte, islas Carolinas y Palaos, a Alemania y las posesiones coloniales españolas se redujeron al Marruecos español, Sahara Español y Guinea española, todo en África. El «

## Alternativa con ChromaDB.

La base de datos ChromaDB, también es muy utilizada para realizar este tipo de tareas.

In [37]:
#!pip install langchain_chroma

In [38]:
import chromadb #pip install chromadb en una terminal
from langchain_chroma import Chroma #pip install langchain_chroma en una terminal

In [ ]:
# Cargar en ChromaDB
#db = Chroma.from_documents(docs, funcion_embedding,collection_name="langchain",persist_directory='./ejemplo_embedding_db')
#Se crean en el directorio persistente la carpeta con los vectores y otra con las string, aparte de una carpeta "index" que mapea vectores y strings
# Fuerzar a guardar los nuevos embeddings en el disco
#db.persist()

## Añadir nueva información a la BD de vectores

In [40]:
# Cargar documento y dividirlo
loader = TextLoader('Fuentes datos/Nuevo_documento.txt', encoding="utf8")
documents = loader.load()

In [41]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

In [ ]:
# Cargar en Chroma
#db = Chroma.from_documents(docs, embedding_function,persist_directory='./ejemplo_embedding_db')
# docs = db.similarity_search('insertar_nueva_búsqueda')

## Comprensión y optimización de resultados a partir de LLMs.

En el apartado anterior hemos visto cómo poder encontrar párrafos de un texto que se asimilan mucho a la consulta que estamos planteando. Pero el resultado que obtenemos no presenta el formato más adecuado para la respuesta que buscamos. En este apartado vamos a ver cómo podemos conseguir esto.

No estamos realizando compresión en el sentido tradicional, sino que utilizamos un LLM para tomar una salida de texto de un documento de mayor tamaño y la limpia / optimiza en una salida más corta y relevante.

In [42]:
from langchain.document_loaders import WikipediaLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import SKLearnVectorStore

In [43]:
#cargamos documentos desde la wikipedia
loader = WikipediaLoader(query='Lenguaje Python',lang="es")
documents = loader.load()

D:\MisTrabajos\IA_generativa\venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file D:\MisTrabajos\IA_generativa\venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Obtenemos de esta manera un documento lo suficientemente grande como para poder trabajar con el para demostrar esta facilidad de LangChain 

In [45]:
len(documents)

24

Procedemos a dividir el documento

In [46]:
# División en fragmentos
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

Created a chunk of size 636, which is longer than the specified 500
Created a chunk of size 515, which is longer than the specified 500
Created a chunk of size 591, which is longer than the specified 500
Created a chunk of size 542, which is longer than the specified 500
Created a chunk of size 653, which is longer than the specified 500
Created a chunk of size 738, which is longer than the specified 500


In [47]:
len(docs)

57

In [48]:
funcion_embedding = OllamaEmbeddings(
    model="llama3.2",
)

In [49]:
persist_path="./BD/ejemplo_wiki_bd"  #ruta donde se guardará la BBDD vectorizada

#Creamos la BBDD de vectores a partir de los documentos y la función embeddings
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=funcion_embedding,
    persist_path=persist_path,
    serializer="parquet", #el serializador o formato de la BD lo definimos como parquet
)

In [50]:
# Fuerza a guardar los nuevos embeddings en el disco
vector_store.persist()

Hacemos una consulta normal de similitud coseno

In [51]:
#Creamos un nuevo documento que será nuestra "consulta" para buscar el de mayor similitud en nuestra Base de Datos de Vectores y devolverlo
consulta = "¿Por qué el lenguaje Python se llama así?"
docs = vector_store.similarity_search(consulta)
print(docs[0].page_content)

Solo cinco tipos de datos básicos
No requiere declaración de variables.
Soporte explícito para programación top-down.
La anidación de instrucciones se indica mediante sangría, a través de la regla de fuera de juego.
Precisión arbitraria, Listas y cadenas de tamaño ilimitado, y otras características que admiten la ortogonalidad y la facilidad de uso para los principiantes.
Como sucede con otros intérpretes, ABC es, además de un lenguaje de programación, un entorno interactivo de trabajo. No requiere de declaraciones de variables, cuenta con el apoyo de la programación top-down. Proporciona una precisión aritmética infinita, ilimitada listas de cadenas, y otras características que da gran facilidad al uso de los principiantes. Sus diseñadores afirman que los programas de ABC son típicamente alrededor de una cuarta parte del tamaño de los programas equivalentes en lenguaje Pascal o en lenguaje C, y además es más legible. 
Originalmente fue una aplicación monolítica, dando lugar a una inca

Como podemos ver la respuesta obtenida ( como antes quizá sin mucho sentido para la pregunta formulada) presenta un aspecto que no es el más adecuado para la presentación a la persona que formula la pregunta. Por ello, a continuación vamos a ver cómo podemos reconducir esto para obtener un resultado  que se adapte más a nuestras pretensiones.

### Consulta con compresión contextual usando LLMs.

Para obtener el resultado pretendido, vamos a importar las siguientes librerías

In [52]:
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [54]:
llm = ChatOpenAI(
    model="llama3.2", #el parámetro temperatura define la aleatoriedad de las respuestas, temperatura = 0 significa el mínimo de aleatoriedad
    temperature=0,
    api_key='ollama', # required, but unused,
) 
compressor = LLMChainExtractor.from_llm(llm)


In [55]:
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vector_store.as_retriever())

In [56]:
compressed_docs = compression_retriever.invoke("¿Por qué el lenguaje Python se llama así?")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: ollama. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
compressed_docs[0].page_content